# Configuring Error Messages

One common challenge with Spark and especially with PySpark is to make sense of error messages. In PySpark, an important part of the problem comes from the fact that two different technologies (JVM and Python) work in conjunction, and both produce stack traces. This notebook tries to provide some guidance for configuring simplified error messages.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","24G") \
        .getOrCreate()

spark

/opt/anaconda3/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 17:34:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/16 17:34:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 1. Provoke an Error

First we provoke an error by creating an ill-formed program

In [2]:
import pandas as pd

replication_df = spark.createDataFrame(pd.DataFrame(list(range(1,1000)),columns=['replication_id'])).repartition(1000, 'replication_id')

In [10]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

outSchema = StructType([StructField('replication_id', IntegerType(), True),
            StructField('sil_score', DoubleType(), True),
            StructField('num_clusters', IntegerType(), True),
            StructField('min_samples', IntegerType(), True),
            StructField('min_cluster_size', IntegerType(), True)])


def run_model(df_pandas: pd.DataFrame) -> pd.DataFrame:
    # Return result as a pandas data frame
    return pd.DataFrame({'replication_id': replication_id, 'sil_score': 2,
                           'num_clusters': 3, 'min_samples': 4,
                           'min_cluster_size': 5}, index=[0])


results = replication_df.groupBy("replication_id").applyInPandas(run_model, outSchema)

# 2. Configuring Error Messages

PySpark provides two important configuration properties `spark.sql.pyspark.jvmStacktrace.enabled` and `spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled` which control how error messages are presented to the developer. We will try different settings and compare the output in the following sections.

## 2.1 Full Detail

First we turn on the JVM Stacktrace and disable a simplification for UDFs. Evil combination, as we will see.

In [11]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",True)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",False)

results.count()

23/11/16 17:38:02 ERROR Executor: Exception in task 27.0 in stage 14.0 (TID 500)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 345, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 86, in dump_stream
    for batch in iterator:
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 338, in 

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 345, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 86, in dump_stream
    for batch in iterator:
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 338, in init_stream_yield_batche

23/11/16 17:38:03 WARN TaskSetManager: Lost task 59.0 in stage 14.0 (TID 532) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:38:03 WARN TaskSetManager: Lost task 65.0 in stage 14.0 (TID 538) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:38:03 WARN TaskSetManager: Lost task 66.0 in stage 14.0 (TID 539) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)


## 2.2 Medium Detail

That was too much. Let's turn off all the JVM Stacktraces, but let's still keep simplification for UDFs turned off. Looks better, but still not perfect.

In [7]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",False)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",False)

results.count()

23/11/16 17:36:04 ERROR Executor: Exception in task 15.0 in stage 11.0 (TID 380)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 345, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 86, in dump_stream
    for batch in iterator:
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 338, in 

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 345, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 86, in dump_stream
    for batch in iterator:
  File "/opt/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 338, in init_stream_yield_batche

23/11/16 17:36:04 WARN TaskSetManager: Lost task 55.0 in stage 11.0 (TID 420) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN TaskSetManager: Lost task 58.0 in stage 11.0 (TID 423) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN TaskSetManager: Lost task 40.0 in stage 11.0 (TID 405) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN TaskSetManager: Lost task 59.0 in stage 11.0 (TID 424) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN TaskSetManager: Lost task 62.0 in stage 11.0 (TID 427) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN TaskSetManager: Lost task 51.0 in stage 11.0 (TID 416) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN TaskSetManager: Lost task 63.0 in stage 11.0 (TID 428) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:36:04 WARN Task

## 2.3 Simplified Detail

Last try: Turn off all the JVM Stacktraces, and enable  simplification for UDFs turned off..

In [6]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",False)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",True)

results.count()

23/11/16 17:35:24 ERROR Executor: Exception in task 66.0 in stage 8.0 (TID 323)]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_10896/3111444347.py", line 17, in run_model
NameError: name 'replication_id' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:118)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.hashAgg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.Generate

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_10896/3111444347.py", line 17, in run_model
NameError: name 'replication_id' is not defined


23/11/16 17:35:24 WARN TaskSetManager: Lost task 11.0 in stage 8.0 (TID 268) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManager: Lost task 1.0 in stage 8.0 (TID 258) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManager: Lost task 19.0 in stage 8.0 (TID 276) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManager: Lost task 36.0 in stage 8.0 (TID 293) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManager: Lost task 18.0 in stage 8.0 (TID 275) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManager: Lost task 21.0 in stage 8.0 (TID 278) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManager: Lost task 20.0 in stage 8.0 (TID 277) (fe49fe119e4f executor driver): TaskKilled (Stage cancelled)
23/11/16 17:35:24 WARN TaskSetManag